In [1]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [ ]:
system_prompt="""
You are an experienced Aircraft Performance Engineer, who has expertise in analyzing aircraft performance data.You are tasked with analyzing the key aircraft performance characteristics data and\
providing a report on which aircraft an airline should choose for thier intercontinental flights.
"""

user_prompt="""
Here is the website content.Please provide your reccomendation on which aircraft an airline should choose for thier intercontinental flights with proper reasoning by considering \
    operating expenses and aircraft performance limitations. Illustrate the aircraft performance comparison in tabular format that would make the information more clear.
"""

In [ ]:
def get_aircraft_performance_review(url1,url2):
    """
    Get aircraft manufacturer details from the given url
    """
    contents1 = fetch_website_contents(url1)
    contents2 = fetch_website_contents(url2)
    messages = [
       {"role": "system", "content": system_prompt},
       {"role": "user", "content": user_prompt + contents1 + contents2} 
    ]

    openai = OpenAI()
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content.strip()

In [ ]:
ac_name1 = input("Enter the aircraft1 family/model: ")
ac_name2 = input("Enter the aircraft2 family/model: ")
if " " in ac_name1:
    ac_name1 = ac_name1.replace(" ", "_")
if " " in ac_name2:
    ac_name2 = ac_name2.replace(" ", "_")
wiki_ac_name1_url = "https://en.wikipedia.org/wiki/" + ac_name1
wiki_ac_name2_url = "https://en.wikipedia.org/wiki/" + ac_name2

print(f"So, you want to compare {ac_name1 } Vs {ac_name2}.")

In [7]:
def review_aircraft_performance(url1,url2):
    review_aircraft_performance= get_aircraft_performance_review(url1,url2)
    display(Markdown(review_aircraft_performance))

In [8]:
review_aircraft_performance(wiki_ac_name1_url,wiki_ac_name2_url)

To provide a recommendation for an airline considering aircraft options for intercontinental flights, I will analyze two popular aircraft models: the Boeing 737 MAX family and the Airbus A320neo family. Since specific performance data is missing from the provided input, I will rely on known characteristics of these aircraft as of October 2023.

### Boeing 737 MAX Overview
**Variants**: 
- 737 MAX 7
- 737 MAX 8
- 737 MAX 9
- 737 MAX 10

**Key Specifications**:
- **Range**: Approximately 3,550 to 4,700 nautical miles depending on the variant.
- **Passenger Capacity**: Ranges from about 138 to 230 passengers.
- **Cruise Speed**: Around Mach 0.79.
- **Fuel Efficiency**: The 737 MAX boasts a 14% lower fuel consumption compared to its predecessor (737 NG) and approximately 20% less fuel use than other aircraft in the same class.

**Strengths**:
- **Cost Efficiency**: Lower operating costs and fuel efficiency make it appealing to airlines.
- **Seat Capacity Flexibility**: Can serve various market demands due to different configurations.

**Weaknesses**:
- **Limited Range**: Though it can serve some transcontinental routes, many intercontinental routes (e.g., across the Atlantic or Pacific) may not be feasible without refueling.
  
### Airbus A320neo Overview
**Variants**:
- A319neo
- A320neo
- A321neo, including A321LR and A321XLR

**Key Specifications**:
- **Range**: The A321XLR variant has an impressive range of up to 4,700 nautical miles, making it suitable for longer-haul flights.
- **Passenger Capacity**: The A320neo family can accommodate between 140 to 244 passengers depending on the specific model and configuration.
- **Cruise Speed**: Similar to the 737 MAX at about Mach 0.78 to 0.80.
- **Fuel Efficiency**: Approximately 15% better fuel efficiency compared to the first generation A320 models, with ongoing improvements in aerodynamic design.

**Strengths**:
- **Longer Range**: The A321XLR can reach destinations that the 737 MAX family may not cover without fuel stops.
- **Greater Passenger Capacity**: The A321LR and A321XLR offer high capacity for more passengers on longer routes.

**Weaknesses**:
- **Higher Operating Costs**: Depending on configuration and flight duration, some operators may find the A320neo family exhibits higher operating costs compared to the smaller 737 MAX variants.

### Recommendation

For an airline looking to perform intercontinental flights efficiently, the **Airbus A321XLR stands out as the more suitable choice** due to its combination of long-range capability, flexible passenger configuration, and improved fuel efficiency compared to the Boeing 737 MAX series.

**Key Reasons**:
1. **Extended Range**: The ability to fly up to 4,700 nautical miles allows for non-stop intercontinental routes which is a critical factor for long haul operations.
2. **Seating Capacity**: The capacity to comfortably seat between 180 to 220 passengers (in varying configurations) helps to accommodate a larger number of travelers.
3. **Operational Flexibility**: The A321XLR allows airlines to target routes that are currently underserved or neglected due to traditional wide-body aircraft limitations, helping to tap into more profitable markets.
4. **Fuel Efficiency**: The latest advancements in the A320neo family, particularly the A321XLR, continue to improve operational costs, making it attractive in competitive intercontinental markets.

In conclusion, the Airbus A321XLR is the recommended aircraft for intercontinental services, providing airlines with the operational flexibility, range, and efficiency necessary for success in a competitive airline industry.